In [1]:
import torch
import copy
import itertools
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from function import Transformer

C:\Users\86153\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 数据

In [2]:
read_data = pd.read_csv('../data/date.csv')
read_data = read_data.set_index('Date')
read_data = read_data.loc['2014-01-01':]
# read_data
fulldata = read_data[[
#                     'AS51',                         #0.431
#                  'CKLSE',                       #0.265
#                  'CSX5P',                   #0.318
                 'DJI',                 #0.43
#                  'FCHI',            #0.17
#                  'FTSE',        #0.58
#                  'GDAXI',   #0.391
#                  'HSI',      #0.374
#                  'IBOVESPA',   #0.298
#                  'IXIC',         #0.524
#                  'KS11',           #0.26
#                  'N225',            #0.621
#                  'RUT',              #0.6
#                  'SENSEX',            #0.282
#                  'SPTSX',               #0.39
#                  'SPX',                   #0.675
#                  'TWII',                    #0.3
#                  'shanghai',                    #0.074
                 'shenzheng']]

In [3]:
# read_data = pd.read_csv('../data/rate/date.csv')
# read_data = read_data.set_index('Date')
# read_data = read_data.loc['2014-01-01':]
# fulldata = read_data[[
# #                         'Australia',
# #                        'Brazil',
# #                       'Canada',
# #                       'EU',
# #                       'Hongkong',
# #                       'India', 
# #                       'Japan',
# #                       'Korea',
# #                       'Malaysia',
# #                       'Taiwan', 
# #                       'UK',
#                       'US' ,
#                       'Close']]

In [4]:
# read_data = pd.read_csv('../data/bond/date.csv')
# read_data = read_data.set_index('Date')
# read_data = read_data.loc['2014-01-01':]
# fulldata = read_data[[
# #                         'Australia',
# #                       'Brazil',
# #                       'Canada',
# #                       'Hongkong',
# #                       'India',
# #                       'Japan',
# #                       'Malaysia',
#                       'US' ,
#                       'Close']]

In [5]:
# a = fulldata[['Close']]
# fulldata = pd.concat([a, a], axis=1)

# 参数设置

In [6]:
predict_day = 1             # 预测未来几天
time_step = 25              # 这个参数很重要，是设置用前多少天的数据来预测，也是LSTM的time step数，请保证训练数据量大于它

In [7]:
#超参数
batch_size = 64  #Batch size
d_model = 512    #Number of embedding dimension
d_ffn = 2048     #Number of Feed Forward Neurals
d_head = 64      #Number of Multi-Head Attention
layers = 16      #Number of Encoding-Decoding layers 
n_heads = 8      #number of heads

In [8]:
c = len(fulldata.columns.tolist())
 # 数据参数
if(c>2):
    feature_columns = list(range(0, c-1))     # 要作为feature的列，按原数据从0开始计算，也可以用list 如 [2,4,6,8] 设置
    label_columns = [c-1]                  # 要预测的列，按原数据从0开始计算, 如同时预测第四，五列 最低价和最高价
else:
    feature_columns = [0]
    label_columns = [1]

shuffle_train_data = True   # 是否对训练数据做shuffle
random_seed = 32            # 随机种子，保证可复现

train_data_rate = 0.80      # 训练数据占总体数据比例，测试数据就是 1-train_data_rate
valid_data_rate = 0.20      # 验证数据占训练数据比例，验证集在训练过程使用，为了做模型和参数选择

# do_continue_train = False    # 每次训练把上一次的final_state作为下一次的init_state，仅用于RNN类型模型，目前仅支持pytorch
do_continue_train = True 
start_num_in_test = 0      # 测试集中前几天的数据会被删掉，因为它不够一个time_step

data, data_column_name = fulldata.values, fulldata.columns.tolist()
data_num = data.shape[0]
train_num = int(data_num * train_data_rate)
std = np.std(data, axis=0)
mean = np.mean(data, axis=0)              # 数据的均值和方差
norm_data = (data - mean)/std   # 归一化，去量纲

In [16]:
# 定义参数范围
parameters = {
    'Batch_size': [32, 64],
    'number_of_heads': [16,32,64,128],
    'Number_of_embedding_dimension': [128,256,512],
    'Number_of_Feed_Forward_Neurals': [512,1024,2048],
    'Number_of_Multi_Head_Attention': [32,64,128,256],
    'Number_of_Encoding_Decoding_layers': [4,6,8,16,32]
}



# 生成所有可能的参数组合
parameter_combinations = []
for batch_size in parameters['Batch_size']:
    for n_heads in parameters['number_of_heads']:
        for d_model in parameters['Number_of_embedding_dimension']:
            for d_ffn in parameters['Number_of_Feed_Forward_Neurals']:
                for d_head in parameters['Number_of_Multi_Head_Attention']:
                    for layers in parameters['Number_of_Encoding_Decoding_layers']:
                        parameter_combinations.append({
                            'Batch_size': batch_size,
                            'number_of_heads': n_heads,
                            'Number_of_embedding_dimension': d_model,
                            'Number_of_Feed_Forward_Neurals': d_ffn,
                            'Number_of_Multi_Head_Attention': d_head,
                            'Number_of_Encoding_Decoding_layers': layers
                })

# #打印所有参数组合
for idx, params in enumerate(parameter_combinations):
    print(f"参数组合 {idx+1}: {params}")

参数组合 1: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16}
参数组合 2: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16}
参数组合 3: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16}
参数组合 4: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16}
参数组合 5: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number

# 初始化函数

In [10]:
def get_train_and_valid_data():
    feature_data = norm_data[:train_num, feature_columns]
#     feature_data = norm_data[:train_num]
    label_data = norm_data[time_step : predict_day + train_num,
                                    label_columns]    # 将延后几天的数据作为label

    
       # 在非连续训练模式下，每time_step行数据会作为一个样本，两个样本错开一行，比如：1-20行，2-21行。。。。
    train_x = [feature_data[i:i+time_step] for i in range(train_num-time_step)]
    train_y = [label_data[i:i+predict_day] for i in range(train_num-time_step)]

    train_x, train_y = np.array(train_x), np.array(train_y)

    train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=valid_data_rate,
                                                              random_state=random_seed,
                                                              shuffle=shuffle_train_data)   # 划分训练和验证集，并打乱
    
    train_X, valid_X, train_Y, valid_Y = train_x, valid_x, train_y, valid_y
    train_and_valid_data=[train_X, train_Y, valid_X, valid_Y]

    train_X, train_Y, valid_X, valid_Y = train_and_valid_data
    train_X, train_Y = torch.from_numpy(train_X).float(), torch.from_numpy(train_Y).float()     # 先转为Tensor
    train_loader = DataLoader(TensorDataset(train_X, train_Y), batch_size=batch_size)    # DataLoader可自动生成可训练的batch数据

    valid_X, valid_Y = torch.from_numpy(valid_X).float(), torch.from_numpy(valid_Y).float()
    valid_loader = DataLoader(TensorDataset(valid_X, valid_Y), batch_size=batch_size)    
    
    
    
    
    return train_loader, valid_loader


In [11]:
def restore_data(pred_result, test_Y):
    # 假设这是数据还原的函数
    # 在这里实现将归一化后的数据还原成原始数据的过程
    # 返回还原后的数据
    std1 = std[1]
    mean1 = mean[1]
    true = test_Y*std1 + mean1
    pred = pred_result*std1 + mean1
    return pred, true


criterion = nn.MSELoss()

def train(model, dataloader, optimizer):
    train_losss = []

    for i, _data in enumerate(train_loader):
        data, label_process = _data[0],_data[1]
        outputs = model(data)
        label_process = torch.squeeze(label_process)
        label_process = label_process.unsqueeze(1) 
        label_process = torch.squeeze(label_process)
        
        outputs = outputs[:,-1]
        if label_process.dim() == 1:
            pass
        else: 
            label_process = label_process[:,-1]
#         print(label_process.shape)
#         print(outputs.shape)
        loss = criterion(outputs, label_process)
        loss_num = loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        train_losss.append(loss.detach().numpy())
    return train_losss


def eval(model, dataloader):    
    pre,true,va_loss = [],[],[]    
    for _valid_X, _valid_Y in valid_loader:
        data, label_process = _valid_X, _valid_Y
        outputs = model(data)
        label_process = torch.squeeze(label_process)
        label_process = label_process.unsqueeze(1) 
        
        label_process = torch.squeeze(label_process)
        
        outputs = outputs[:,-1]
        if label_process.dim() == 1:
            pass
        else: 
            label_process = label_process[:,-1]
        loss = criterion(outputs, label_process)
        loss_num = loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        va_loss.append(loss_num)
        pre.append(outputs.cpu().detach().numpy())
        true.append(label_process.cpu().detach().numpy())
    valid_loss_cur = np.mean(va_loss)
    return valid_loss_cur, true, pre

In [12]:
 # 网络参数
feature = norm_data.shape[1]-1
# feature = norm_data.shape[1]
predict_steps = predict_day
epochs = 2 # 整个训练集被训练多少遍，不考虑早停的前提下
patience = 70               # 训练多少epoch，验证集没提升就停掉
bad_epoch = 0
results = []

# for i in range(len(parameter_combinations)):
for i in range(3):
        # 假设你想选择第三个参数组合
    selected_params = parameter_combinations[i]

    # 然后你就可以按照键来获取对应的值
    batch_size = selected_params['Batch_size']
    n_heads = selected_params['number_of_heads']
    d_model = selected_params['Number_of_embedding_dimension']
    d_ffn = selected_params['Number_of_Feed_Forward_Neurals']
    d_head = selected_params['Number_of_Multi_Head_Attention']
    layers = selected_params['Number_of_Encoding_Decoding_layers']
    
    train_loader, valid_loader = get_train_and_valid_data()
    model = Transformer(d_model=d_model,
                    d_head=d_head,
                    n_heads=n_heads,
                    d_ffn=d_ffn,
                    layers=layers,
                    feature=feature,
                    predict_steps=predict_steps)
    optimizer = optim.SGD(model.parameters(), lr=7e-3, momentum=0.99)
    best_loss = 5
    vl_loss = []

    for epoch_i in range(epochs):


        train_loss = train(model, train_loader, optimizer)

        eval_loss, tru, pre = eval(model, valid_loader)

        vl_loss.append(eval_loss)
        if eval_loss < best_loss:
            best_loss = eval_loss
            best_epoch = epoch_i
            best_result_valid = pre
            best_true_valid = tru
    #         torch.save(model.state_dict(), '../model/best_ANN_trainModel.pth')  # 模型保存
        else:
            bad_epoch += 1
            if bad_epoch >= patience:    # 如果验证集指标连续patience个epoch没有提升，就停掉训练
                break
    results.append({
                    'epoch': best_epoch,
                    'loss': best_loss})

In [13]:
for idx, params in enumerate(results):
    print(f"参数组合 {idx+1}: {params}")

参数组合 1: {'epoch': 1, 'loss': 0.8788063360585107}
参数组合 2: {'epoch': 0, 'loss': 1.401423262225257}
参数组合 3: {'epoch': 0, 'loss': 0.9472353259722391}


In [14]:
# 合并参数组合和结果到一个字典列表
combined_params = []
for params, result in zip(parameter_combinations, results):
    combined_params.append({**params, **result})

In [15]:
# 根据损失值排序
sorted_params = sorted(combined_params, key=lambda x: x['loss'])

# 取前三个
top_three_params = sorted_params[:10]

print("Top Three Parameters:")
for i, param in enumerate(top_three_params, start=1):
    print(f"Rank {i}: {param}")

Top Three Parameters:
Rank 1: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16, 'epoch': 1, 'loss': 0.8788063360585107}
Rank 2: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16, 'epoch': 0, 'loss': 0.9472353259722391}
Rank 3: {'Batch_size': 32, 'number_of_heads': 8, 'Number_of_embedding_dimension': 512, 'Number_of_Feed_Forward_Neurals': 2048, 'Number_of_Multi_Head_Attention': 64, 'Number_of_Encoding_Decoding_layers': 16, 'epoch': 0, 'loss': 1.401423262225257}
